# Importing the necessary libraries and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df_patients = pd.read_csv('DA-data-analysis-assignment-dataset/patients.csv')
df_transactions = pd.read_csv('DA-data-analysis-assignment-dataset/transactions.csv')

## Data Cleaning
### df_patients

In [2]:
df_patients.head()

,Patient ID,Gender,Birthday
0,10003.0,F,"June 14, 1995"
1,10006.0,F,"February 17, 1947"
2,10026.0,F,"September 20, 1977"
3,10027.0,M,"May 16, 1987"
4,10030.0,F,"May 12, 1987"


In [3]:
print(f"'df_patients' contains {df_patients.shape[0]} rows with {df_patients.shape[1]} columns")

'df_patients' contains 1643 rows with 3 columns


In [4]:
df_patients.duplicated().sum()

0

In [5]:
df_patients['Patient ID'].duplicated().sum()

134

In [6]:
df_patients[df_patients['Patient ID'].duplicated()].head()

,Patient ID,Gender,Birthday
1509,NaN,F,"July 30, 1975"
1510,NaN,F,"January 8, 1995"
1511,NaN,F,"December 12, 1985"
1512,NaN,F,"May 9, 1978"
1513,NaN,F,"March 24, 1990"


All duplicated 'patient ID' are due to __NaN__ value

### Missing values

In [7]:
def chk_na(df):
    '''
    Scan a DataFrame for missing values.
    
    Parameters :
    - df (DataFrame): Dataframe containing the data.
    
    Returns :
    DataFrame: A DataFrame containing two columns:
        - 'cnt_missing': Number of items missing per column.
        - 'percentage_missing': Percentage of items missing per column.
    '''
    
    df_perc = pd.DataFrame(df.isnull().sum()/len(df), columns=['percentage_missing'])
    df_cnt = pd.DataFrame(df.isnull().sum(), columns=['cnt_missing'])
    return pd.concat([df_cnt, df_perc], axis=1)
    
chk_na(df_patients)

,cnt_missing,percentage_missing
Patient ID,135,0.082167
Gender,0,0.000000
Birthday,0,0.000000


'Patient ID' unstructurally missing

In [8]:
df_transactions[~df_transactions['Patient ID'].isin(df_patients['Patient ID'])]

,Consult ID,Doctor ID,Clinic Location,Patient ID,Appointment Date Time,Appointment Type,Appointment Length,Price,Reason For Visit,Payment Mode,Visit Status
1296,10882,813668,West,11042,1/25/22 2:00 PM,Follow-up Visit,15,70,Plantar warts,Credit Card,Completed
1459,10914,813668,West,17060,1/27/22 2:45 PM,Follow-up Visit,15,70,Heel pain,Credit Card,Completed


So even though, we have 135 missing 'Patient ID', we really only missing 2 row of data to link to 'df_transactions'. In a nutshell, only 2 consult we can't get the 'Gender' & 'Birthday' of the patient.

## df_transactions

In [9]:
df_transactions.head()

,Consult ID,Doctor ID,Clinic Location,Patient ID,Appointment Date Time,Appointment Type,Appointment Length,Price,Reason For Visit,Payment Mode,Visit Status
0,10001,881452,North,16180,1/3/22 8:00 AM,New Patient,30,80,Foot pain,Cash,No-Show
1,10327,745113,South,14257,1/3/22 8:00 AM,Follow-up Visit,30,70,Wound care,Insurance,Completed
2,10635,813668,West,10964,1/3/22 8:00 AM,New Patient,30,80,Ankle sprain,Credit Card,Completed
3,11322,871346,North East,18906,1/3/22 8:00 AM,Follow-up Visit,15,70,Injection,Cash,Rescheduled
4,10954,696455,East,11360,1/3/22 8:15 AM,Follow-up Visit,15,70,Nail care,Credit Card,Completed


In [10]:
print(f"'df_transactions' contains {df_transactions.shape[0]} rows with {df_transactions.shape[1]} columns")

'df_transactions' contains 1645 rows with 11 columns


In [11]:
df_transactions.duplicated().sum()

0

In [12]:
df_transactions['Consult ID'].duplicated().sum()

0

### Missing values

In [13]:
chk_na(df_transactions)

,cnt_missing,percentage_missing
Consult ID,0,0.0
Doctor ID,0,0.0
Clinic Location,0,0.0
Patient ID,0,0.0
Appointment Date Time,0,0.0
Appointment Type,0,0.0
Appointment Length,0,0.0
Price,0,0.0
Reason For Visit,0,0.0
Payment Mode,0,0.0


### Understanding the features

In [14]:
cat_feat = df_transactions.drop('Appointment Date Time', axis=1).select_dtypes(include=object).columns
df_cat_feat = pd.DataFrame(columns=['Column', 'Nunique', 'Values'])

for col in cat_feat:
    df_cat_feature2 = pd.DataFrame({'Column': col,  
                                    'Nunique' : df_transactions[col].nunique(), 
                                    'Values' : [df_transactions[col].unique()]})
    
    df_cat_feat = pd.concat([df_cat_feat, df_cat_feature2])

df_cat_feat

,Column,Nunique,Values
0,Clinic Location,5,"[North, South, West, North East, East]"
0,Appointment Type,3,"[New Patient, Follow-up Visit, Procedure]"
0,Reason For Visit,37,"[Foot pain, Wound care, Ankle sprain, Injection, Nail care, Toe infection, Plantar fasciitis, Surgical consult, Routine foot care, Achilles tendon, Calluses, Bunion, Big toe pain, Plantar warts, Heel pain, Flat feet, Ankle pain, Preop consult, Ingrown toenail, Rash, Postop, Orthotic F/U, Arch pain, Athlete's foot, Nail fungus, Sports injury, Hammertoes, Abnormal gait, Wound, Toe pain, Toe walking, Biopsy, Leg deformity, Neuroma, Procedure F/U, Leg pain, Foot infection]"
0,Payment Mode,3,"[Cash, Insurance, Credit Card]"
0,Visit Status,4,"[No-Show, Completed, Rescheduled, Cancelled]"


In [15]:
print(f"'Doctor ID' suggests {df_transactions['Doctor ID'].nunique()} unique doctor")

'Doctor ID' suggests 5 unique doctor


In [16]:
df_transactions['Appointment Date Time'] = pd.to_datetime(df_transactions['Appointment Date Time'], format='%m/%d/%y %I:%M %p')

print(f"Data range from {df_transactions['Appointment Date Time'].min()} to {df_transactions['Appointment Date Time'].max()}")

Data range from 2022-01-03 08:00:00 to 2022-01-31 15:30:00


In [17]:
df_transactions['Appointment Length'].value_counts()

Appointment Length
15    969
30    613
60     63
Name: count, dtype: int64

In [18]:
df_transactions['Price'].value_counts()

Price
70     1121
80      461
100      63
Name: count, dtype: int64

### Clinic operating hours

In [19]:
df_transactions['Appointment DOW'] = df_transactions['Appointment Date Time'].dt.strftime('%A')
df_transactions['hour'] = df_transactions['Appointment Date Time'].dt.hour
df_transactions['Appointment DOW'] = pd.Categorical(df_transactions['Appointment DOW'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], ordered=True)

pt = df_transactions.pivot_table(index='Appointment DOW', columns='hour', aggfunc='size')

pt

hour,8,9,10,11,12,13,14,15,23
Appointment DOW,,,,,,,,,
Monday,55,43,45,45,33,59,54,46,2
Tuesday,38,38,44,38,28,48,51,27,2
Wednesday,33,36,34,27,28,55,50,29,0
Thursday,55,46,54,36,25,61,43,20,0
Friday,46,45,45,39,26,51,41,24,0


Data suggests that clinic likely operate Monday to Friday, 8am to 3pm. <br><br> <b>RE : 2 consultation happened after 11pm :</b><br> - I've spoken to the consulting doctor (696455), who confirmed veracity of the data. <br>- He have explained that due to our operating hours, there're many patients who wants our service but were unable to visit.</b>- These 4 patients he have made special concession to as they're his old patients who followed him to our clinic when he joined us <br><br> <b>Business Opportunity :</b><br> To capture this group of patients, we can consider to either to start a evening or weekend clinic or even both. To be prudent, I suggest we start with collecting feedback from patients when they contact us for appointment, check and record their interest for evening or weekend clinic and what time they would most likely want to visit if they show interest.<br><br> After we have studied the data, then we can start small scale (1 weekday night or 1 weekend) and decide if the initiative warrants expansion or scrape altogether depending on the impact on revenue.

### Logic to differentiate New/Returning Patient
Typically, we would take the <b>first completed consultation</b> of the patient and labelled that consult to be a <b>New Patient</b> and any subsequent consultation to be <b>Returning Patient</b> but this will not work well on our data set because we only have 1 month data and not full historical data. <br> So, for this data set, I'm gonna rely on `Appointment Type` to help me identify <b>New</b> and <b>Returning</b> <br><br>  if `Appointment Type`=='New Patient' THEN 'New' ELSE 'Returning' <br><br> Let's begin by checking if data in `Appointment Type` is reliable enough.

In [20]:
new = df_transactions[(df_transactions['Appointment Type']=='New Patient')].groupby('Patient ID').agg('size').reset_index()

df_transactions[df_transactions['Patient ID'].isin(new[new[0]>1]['Patient ID'])].sort_values(by='Patient ID').reset_index()

,index,Consult ID,Doctor ID,Clinic Location,Patient ID,Appointment Date Time,Appointment Type,Appointment Length,Price,Reason For Visit,Payment Mode,Visit Status,Appointment DOW,hour
0,243,11374,871346,North East,11617,2022-01-05 14:45:00,New Patient,30,80,Hammertoes,Credit Card,Completed,Wednesday,14
1,1249,10560,745113,North,11617,2022-01-25 09:00:00,New Patient,30,80,Wound,Insurance,Completed,Tuesday,9
2,499,10726,813668,West,11851,2022-01-10 15:00:00,New Patient,30,80,Sports injury,Credit Card,Completed,Monday,15
3,598,10740,813668,West,11851,2022-01-11 15:30:00,New Patient,30,80,Heel pain,Credit Card,Completed,Tuesday,15
4,1627,11319,696455,East,13002,2022-01-31 14:15:00,New Patient,30,80,Calluses,Credit Card,Completed,Monday,14
5,1203,11567,871346,North East,13002,2022-01-24 13:00:00,New Patient,30,80,Routine foot care,Insurance,Completed,Monday,13
6,1116,11208,696455,East,13120,2022-01-21 11:30:00,New Patient,30,80,Routine foot care,Insurance,Completed,Friday,11
7,1094,10216,881452,North,13120,2022-01-21 09:30:00,New Patient,30,80,Neuroma,Credit Card,Completed,Friday,9
8,1457,10599,745113,South,13474,2022-01-27 14:30:00,Follow-up Visit,30,70,Wound care,Insurance,Completed,Thursday,14
9,935,11164,696455,South,13474,2022-01-19 10:30:00,New Patient,30,80,Heel pain,Credit Card,Completed,Wednesday,10


Looks like we have several patients who have more than 1 `Appointment Type` labelled as <b>'New Patient'</b>. Let's clean this by update any subsequent consultations to be <b>Follow-up Visit</b>

In [21]:
df_transactions['First_visit_date'] = df_transactions.groupby('Patient ID')['Appointment Date Time'].transform('min')

df_transactions['Appointment Type'] = np.where((df_transactions['Appointment Date Time']!=df_transactions['First_visit_date'])&(df_transactions['Appointment Type']=='New Patient'), 'Follow-up Visit', df_transactions['Appointment Type'])

df_transactions[df_transactions['Patient ID'].isin([11617, 11851, 13002])].sort_values(by='Patient ID').reset_index()

,index,Consult ID,Doctor ID,Clinic Location,Patient ID,Appointment Date Time,Appointment Type,Appointment Length,Price,Reason For Visit,Payment Mode,Visit Status,Appointment DOW,hour,First_visit_date
0,243,11374,871346,North East,11617,2022-01-05 14:45:00,New Patient,30,80,Hammertoes,Credit Card,Completed,Wednesday,14,2022-01-05 14:45:00
1,1249,10560,745113,North,11617,2022-01-25 09:00:00,Follow-up Visit,30,80,Wound,Insurance,Completed,Tuesday,9,2022-01-05 14:45:00
2,499,10726,813668,West,11851,2022-01-10 15:00:00,New Patient,30,80,Sports injury,Credit Card,Completed,Monday,15,2022-01-10 15:00:00
3,598,10740,813668,West,11851,2022-01-11 15:30:00,Follow-up Visit,30,80,Heel pain,Credit Card,Completed,Tuesday,15,2022-01-10 15:00:00
4,1203,11567,871346,North East,13002,2022-01-24 13:00:00,New Patient,30,80,Routine foot care,Insurance,Completed,Monday,13,2022-01-24 13:00:00
5,1627,11319,696455,East,13002,2022-01-31 14:15:00,Follow-up Visit,30,80,Calluses,Credit Card,Completed,Monday,14,2022-01-24 13:00:00


Yup, better now~

### Computing Time Loss

In [22]:
df_transactions_copy = df_transactions.copy()

non_completed = df_transactions_copy[df_transactions_copy['Visit Status']!='Completed']
non_completed = non_completed.merge(df_transactions_copy[(df_transactions_copy['Visit Status']=='Completed')][['Clinic Location', 'Appointment Date Time', 'Consult ID', 'Visit Status', 'Appointment Length']], how='inner', on=['Clinic Location', 'Appointment Date Time'])

non_completed = non_completed[['Consult ID_x', 'Clinic Location', 'Appointment Date Time', 'Appointment Length_x', 'Visit Status_x', 'Consult ID_y', 'Visit Status_y', 'Appointment Length_y']]
non_completed.columns = ['Old Consult ID', 'Clinic Location', 'Appointment Date Time', 'Old Appointment Length', 'Old Visit Status', 'Replaced Consult ID', 'Replaced Visit Status', 'Replaced Appointment Length']
non_completed = non_completed[['Appointment Date Time', 'Clinic Location', 'Old Consult ID', 'Old Appointment Length', 'Old Visit Status', 'Replaced Consult ID', 'Replaced Appointment Length', 'Replaced Visit Status']]


non_completed

,Appointment Date Time,Clinic Location,Old Consult ID,Old Appointment Length,Old Visit Status,Replaced Consult ID,Replaced Appointment Length,Replaced Visit Status
0,2022-01-03 12:45:00,North,10014,15,Rescheduled,10965,15,Completed
1,2022-01-13 14:00:00,East,11121,15,Rescheduled,10156,30,Completed
2,2022-01-20 08:30:00,West,10827,15,Cancelled,10194,15,Completed
3,2022-01-21 08:00:00,North East,11197,15,No-Show,11540,30,Completed
4,2022-01-21 10:45:00,North,10531,15,Cancelled,10220,15,Completed


Looks like we have 5 consultation in Jan-22, where the non-completed slots were taken up by another patient. When computing `time_loss`, I recommend to exclude these instances. Otherwise any effort to fill these non-completed slot would not be visible

In [23]:
non_completed = df_transactions[df_transactions['Visit Status']!='Completed'][['Consult ID', 'Clinic Location', 'Appointment Length', 'Appointment Date Time']].reset_index(drop=1)
non_completed.columns = ['Consult ID', 'Clinic Location', 'time_loss', 'Appointment Date Time']
non_completed


df_transactions2 = df_transactions.copy()

non_completed2 = non_completed.copy()
non_completed2 = non_completed2.merge(df_transactions[df_transactions['Visit Status']=='Completed'][['Clinic Location', 'Appointment Date Time', 'Appointment Length']], on=['Clinic Location', 'Appointment Date Time'], how='left').fillna(0)
non_completed2['actual_time_loss'] = np.where(non_completed2['time_loss']-non_completed2['Appointment Length']<=0, 0, non_completed2['time_loss']-non_completed2['Appointment Length']) 

df_transactions2 = df_transactions2.merge(non_completed2[['Consult ID', 'actual_time_loss']], on='Consult ID', how='left')
df_transactions2['actual_time_loss'] = df_transactions2['actual_time_loss'].fillna(0)

df_transactions2

,Consult ID,Doctor ID,Clinic Location,Patient ID,Appointment Date Time,Appointment Type,Appointment Length,Price,Reason For Visit,Payment Mode,Visit Status,Appointment DOW,hour,First_visit_date,actual_time_loss
0,10001,881452,North,16180,2022-01-03 08:00:00,New Patient,30,80,Foot pain,Cash,No-Show,Monday,8,2022-01-03 08:00:00,30.0
1,10327,745113,South,14257,2022-01-03 08:00:00,Follow-up Visit,30,70,Wound care,Insurance,Completed,Monday,8,2022-01-03 08:00:00,0.0
2,10635,813668,West,10964,2022-01-03 08:00:00,New Patient,30,80,Ankle sprain,Credit Card,Completed,Monday,8,2022-01-03 08:00:00,0.0
3,11322,871346,North East,18906,2022-01-03 08:00:00,Follow-up Visit,15,70,Injection,Cash,Rescheduled,Monday,8,2022-01-03 08:00:00,15.0
4,10954,696455,East,11360,2022-01-03 08:15:00,Follow-up Visit,15,70,Nail care,Credit Card,Completed,Monday,8,2022-01-03 08:15:00,0.0
5,11323,871346,North East,15497,2022-01-03 08:15:00,Follow-up Visit,15,70,Toe infection,Insurance,Completed,Monday,8,2022-01-03 08:15:00,0.0
6,10002,881452,North,16943,2022-01-03 08:30:00,Follow-up Visit,15,70,Plantar fasciitis,Credit Card,Completed,Monday,8,2022-01-03 08:30:00,0.0
7,10328,745113,South,18025,2022-01-03 08:30:00,Follow-up Visit,30,70,Wound care,Insurance,Completed,Monday,8,2022-01-03 08:30:00,0.0
8,10636,813668,South,13474,2022-01-03 08:30:00,New Patient,30,80,Surgical consult,Credit Card,Completed,Monday,8,2022-01-03 08:30:00,0.0
9,10955,696455,East,13615,2022-01-03 08:30:00,Follow-up Visit,15,70,Routine foot care,Insurance,Completed,Monday,8,2022-01-03 08:30:00,0.0


In [24]:
combine = df_transactions2.merge(df_patients, how='left', on='Patient ID')
combine.to_excel('combine.xlsx', index=False)